In [1]:
import pandas as pd

PRED_PATH = "../data/processed/nfl_to_nfl_all_predictions_2026_named.csv"  # or your non-named file
SCORE_COL = "pred_fp_ppr_2026"

# 1) Load predictions
preds = pd.read_csv(PRED_PATH)

# 2) Keep only the columns we need (extra columns are fine; this is just clean)
keep_cols = [c for c in ["player_id", "player_name", "position", SCORE_COL] if c in preds.columns]
preds = preds[keep_cols].copy()

# 3) Sort within position so "best available" is always first
preds = preds.sort_values(["position", SCORE_COL], ascending=[True, False])

# 4) Build pools: dict[position] -> list of player_ids in ranked order
pools = {
    pos: group["player_id"].tolist()
    for pos, group in preds.groupby("position", sort=False)
}

# 5) Quick sanity
for pos in ["QB", "RB", "WR", "TE"]:
    if pos in pools:
        print(pos, "pool size:", len(pools[pos]), "top player_id:", pools[pos][0])

QB pool size: 81 top player_id: 00-0034857
RB pool size: 160 top player_id: 00-0033280
WR pool size: 240 top player_id: 00-0039075
TE pool size: 138 top player_id: 00-0037744


In [ ]:


SCORE_COL = "pred_fp_ppr_2026"


# Build a fast lookup so we can get name/score instantly
preds_idx = preds.set_index("player_id")

def recommend_one(position: str):
    """Return the top available player for a position, or None if pool empty."""
    position = position.upper().strip()
    if position not in pools or len(pools[position]) == 0:
        return None

    player_id = pools[position][0]
    row = preds_idx.loc[player_id]

    return {
        "player_id": player_id,
        "player_name": row["player_name"] if "player_name" in row.index else None,
        "position": row["position"],
        "pred_fp_ppr_2026": float(row[SCORE_COL]),
    }

def remove_from_pool(position: str, player_id: str) -> bool:
    """Remove a player from a position pool. Returns True if removed, False if not found."""
    position = position.upper().strip()
    if position not in pools:
        return False
    try:
        pools[position].remove(player_id)
        return True
    except ValueError:
        return False

def add_back_to_pool(position: str, player_id: str) -> bool:
    """
    Add a player back into a position pool in the correct ranked spot.
    Returns True if inserted, False if already present or not valid for that position.
    """
    position = position.upper().strip()
    if position not in pools:
        return False
    if player_id in pools[position]:
        return False
    if player_id not in preds_idx.index:
        return False

    # Ensure the player belongs to this position
    if str(preds_idx.loc[player_id]["position"]).upper() != position:
        return False

    # Insert back based on their global rank order for that position
    # We'll use the prediction score to find insertion point (descending).
    score = float(preds_idx.loc[player_id][SCORE_COL])

    pool_ids = pools[position]
    insert_at = len(pool_ids)  # default: end
    for i, pid in enumerate(pool_ids):
        pid_score = float(preds_idx.loc[pid][SCORE_COL])
        if score > pid_score:
            insert_at = i
            break

    pools[position].insert(insert_at, player_id)
    return True


# --- tiny sanity demo (optional) ---
# rec = recommend_one("RB")
# print(rec)
# remove_from_pool("RB", rec["player_id"])
# print("next:", recommend_one("RB"))
# add_back_to_pool("RB", rec["player_id"])
# print("restored:", recommend_one("RB"))

In [3]:

SCORE_COL = "pred_fp_ppr_2026"

# Assumes you already have:
# preds (DataFrame), pools (dict), preds_idx (index by player_id),
# and functions: recommend_one, remove_from_pool, add_back_to_pool

VALID_POS = {"QB", "RB", "WR", "TE"}

def get_top_n(position: str, n: int = 10) -> pd.DataFrame:
    """Return top N available players for a position (no mutation)."""
    position = position.upper().strip()
    ids = pools.get(position, [])[:n]
    if not ids:
        return pd.DataFrame(columns=["rank", "player_id", "player_name", "pred_fp_ppr_2026"])

    rows = preds_idx.loc[ids].copy()
    # if single row comes back as Series, make it DF
    if isinstance(rows, pd.Series):
        rows = rows.to_frame().T

    out = rows.reset_index().rename(columns={"index": "player_id"})
    if "player_name" not in out.columns:
        out["player_name"] = None

    out = out[["player_id", "player_name", SCORE_COL]]
    out = out.rename(columns={SCORE_COL: "pred_fp_ppr_2026"})
    out.insert(0, "rank", range(1, len(out) + 1))
    return out

def draft_assistant_cli():
    print("\n=== Draft Pick Assistant (CLI) ===")
    print("Type QB/RB/WR/TE to get a recommendation. Type 'quit' to exit.\n")

    while True:
        pos = input("Choose a position (QB/RB/WR/TE): ").strip().upper()
        if pos in {"QUIT", "EXIT", "Q"}:
            print("Good luck in your draft ✌️")
            break
        if pos not in VALID_POS:
            print("Invalid position. Please enter QB, RB, WR, or TE.\n")
            continue

        # Recommend until accepted or user chooses to stop
        while True:
            rec = recommend_one(pos)
            if rec is None:
                print(f"No players left in pool for {pos}.\n")
                break

            name = rec["player_name"] or "(name unavailable)"
            score = rec["pred_fp_ppr_2026"]
            pid = rec["player_id"]

            print(f"\nRecommendation for {pos}: {name} (player_id={pid})")
            print(f"Predicted fantasy points (PPR): {score:.2f}")

            decision = input("Accept this recommendation? (y/n): ").strip().lower()
            if decision in {"y", "yes"}:
                # Treat accepted as "picked" => remove from pool
                remove_from_pool(pos, pid)
                print(f"✅ Locked in: {name}\n")
                break

            if decision not in {"n", "no"}:
                print("Please answer y or n.")
                continue

            # Denied: ask why
            reason = input("Why deny? (1=player already picked, 2=I disagree): ").strip()
            if reason == "1":
                # remove and immediately suggest next
                removed = remove_from_pool(pos, pid)
                if removed:
                    print(f"Removed {name} from {pos} pool (already picked). Trying next best...")
                else:
                    print("Couldn't remove player (not found), trying next best...")
                continue

            elif reason == "2":
                # do NOT remove; show top 10 options in order (including the recommended player)
                top10 = get_top_n(pos, n=10)
                print("\nTop 10 available suggestions:")
                if top10.empty:
                    print("(none available)\n")
                else:
                    # nice print
                    for _, r in top10.iterrows():
                        nm = r["player_name"] or "(name unavailable)"
                        print(f"{int(r['rank']):>2}. {nm} (id={r['player_id']}) — {float(r['pred_fp_ppr_2026']):.2f}")
                    print()

                # After showing options, let them either accept one by id, or go back
                choice = input("Enter a player_id to accept, or press Enter to go back: ").strip()
                if choice == "":
                    # go back to recommending (still same top player unless they remove it)
                    continue

                # Accept chosen id (only if in pool)
                if choice not in pools.get(pos, []):
                    print("That player_id is not currently available in this position pool.\n")
                    continue

                chosen_row = preds_idx.loc[choice]
                chosen_name = chosen_row["player_name"] if "player_name" in chosen_row.index else "(name unavailable)"
                remove_from_pool(pos, choice)
                print(f"✅ Locked in: {chosen_name}\n")
                break

            else:
                print("Please enter 1 or 2.\n")
                continue

# Run it
# draft_assistant_cli()

In [ ]:
import json
from pathlib import Path

STATE_PATH = Path("../data/processed/draft_state.json")

def reset_state_from_preds():
    """Rebuild pools from preds (ranked order) and clear picked."""
    fresh_pools = {
        pos: group["player_id"].tolist()
        for pos, group in preds.sort_values(["position", SCORE_COL], ascending=[True, False]).groupby("position", sort=False)
    }
    picked = {pos: [] for pos in VALID_POS}
    return fresh_pools, picked

def save_state(pools, picked, path=STATE_PATH):
    path.parent.mkdir(parents=True, exist_ok=True)
    payload = {"pools": pools, "picked": picked}
    path.write_text(json.dumps(payload, indent=2))

def load_state(path=STATE_PATH):
    if not path.exists():
        return None
    payload = json.loads(path.read_text())
    return payload.get("pools"), payload.get("picked")

# ---- Load state on startup (or initialize) ----
_loaded = load_state()
if _loaded is None:
    pools, picked = reset_state_from_preds()
    save_state(pools, picked)
else:
    pools, picked = _loaded

def pool_size_line():
    return " | ".join([f"{pos}: {len(pools.get(pos, []))} left" for pos in ["QB","RB","WR","TE"]])

# ---- Update CLI to include reset + persistence ----
def draft_assistant_cli():
    print("\n=== Draft Pick Assistant (CLI) ===")
    print("Type QB/RB/WR/TE to get a recommendation.")
    print("Commands: 'reset' (restart pools), 'status' (show remaining), 'quit'\n")

    while True:
        cmd = input(f"({pool_size_line()})\nChoose position or command: ").strip().upper()

        if cmd in {"QUIT", "EXIT", "Q"}:
            save_state(pools, picked)
            print("Saved draft state. Good luck ✌️")
            break

        if cmd == "STATUS":
            print(pool_size_line(), "\n")
            continue

        if cmd == "RESET":
            pools_new, picked_new = reset_state_from_preds()
            pools.clear(); pools.update(pools_new)
            picked.clear(); picked.update(picked_new)
            save_state(pools, picked)
            print("✅ Draft state reset.\n")
            continue

        pos = cmd
        if pos not in VALID_POS:
            print("Invalid input. Enter QB/RB/WR/TE or a command.\n")
            continue

        while True:
            rec = recommend_one(pos)
            if rec is None:
                print(f"No players left in pool for {pos}.\n")
                break

            name = rec["player_name"] or "(name unavailable)"
            score = rec["pred_fp_ppr_2026"]
            pid = rec["player_id"]

            print(f"\nRecommendation for {pos}: {name} (player_id={pid})")
            print(f"Predicted fantasy points (PPR): {score:.2f}")

            decision = input("Accept this recommendation? (y/n): ").strip().lower()
            if decision in {"y", "yes"}:
                remove_from_pool(pos, pid)
                picked[pos].append(pid)
                save_state(pools, picked)
                print(f"✅ Locked in: {name}\n")
                break

            if decision not in {"n", "no"}:
                print("Please answer y or n.")
                continue

            reason = input("Why deny? (1=player already picked, 2=I disagree): ").strip()
            if reason == "1":
                # remove from pool permanently
                remove_from_pool(pos, pid)
                picked[pos].append(pid)
                save_state(pools, picked)
                print(f"Removed {name} from {pos} pool (already picked). Trying next best...")
                continue

            if reason == "2":
                top10 = get_top_n(pos, n=10)
                print("\nTop 10 available suggestions:")
                if top10.empty:
                    print("(none available)\n")
                else:
                    for _, r in top10.iterrows():
                        nm = r["player_name"] or "(name unavailable)"
                        print(f"{int(r['rank']):>2}. {nm} (id={r['player_id']}) — {float(r['pred_fp_ppr_2026']):.2f}")
                    print()

                choice = input("Enter a player_id to accept, or press Enter to go back: ").strip()
                if choice == "":
                    continue

                if choice not in pools.get(pos, []):
                    print("That player_id is not currently available in this position pool.\n")
                    continue

                chosen_row = preds_idx.loc[choice]
                chosen_name = chosen_row["player_name"] if "player_name" in chosen_row.index else "(name unavailable)"
                remove_from_pool(pos, choice)
                picked[pos].append(choice)
                save_state(pools, picked)
                print(f"✅ Locked in: {chosen_name}\n")
                break

            print("Please enter 1 or 2.\n")

# Run it


In [ ]:
draft_assistant_cli()


=== Draft Pick Assistant (CLI) ===
Type QB/RB/WR/TE to get a recommendation.
Commands: 'reset' (restart pools), 'status' (show remaining), 'quit'

